In [5]:
# 导入相关库
import numpy as np
import pandas as pd
import time

np.random.seed(2)
# 伪随机数列，确定的随机种子可以使得随机数列一致

In [6]:
# 定义超参数
N_STATES = 6
# 状态的总数，一个一维世界
ACTIONS = ['left', 'right']
# 动作
EPSILON = 0.9
# greedy policy 属于Q_learning的一种方法，即90%选择最优策略，10%不选择
ALPHA = 0.1
# learning rate
LAMBDA = 0.9
# 衰减因子，未来的奖励比不上现在的奖励，所以有衰减
MAX_EPISODES = 13
# 最大回合数
FRESH_TIME = 0.3
# 走一步的时间，为了方便观察游戏而已

In [7]:
# 函数定义
def build_q_table(n_states, actions):
    table = pd.DataFrame(
        np.zeros((n_states, len(actions))),
        # 初始值，直接在pd.DataFrame中放一个数组
        columns=actions,
        # 列索引为actions
    )
    # 用DataFrame存储q表格
    return table
# 输入状态数、动作，分别作为行、列标，内容填充该状态下做该动作的价值，初始化为0

def choose_action(state, q_table):
    # 根据所处状态及做出动作带来的奖励进行动作选择
    state_actions = q_table.iloc[state, :]
    # iloc函数：根据标签的所在位置，从0开始计数，先选取行再选取列
    # 实际上就是通过定位取出特定行、列中的内容
    if (np.random.uniform() > EPSILON) or (state_actions.all() == 0):
        #  numpy.random.uniform(low,high,size)
        # low: 采样下界，float类型，默认值为0；
        # high: 采样上界，float类型，默认值为1；
        # size: 输出样本数目，为int或元组(tuple)类型，例如，size=(m,n,k), 则输出 m * n * k 个样本，缺省时输出1个值。
        # or (state_actions.all() == 0),初始化问题，刚开始的时候都是0，这时候随机选择一个动作
        action_name = np.random.choice(ACTIONS)
        # 随机选择一个动作
    else:
        # act greedy，选取价值最高的动作
        action_name = state_actions.idxmax()
        # 原本是.argmax, 从中选取最大的
        # 将argmax替换为idxmax，因为argmax在新版本的panda中意味着不同的功能
        # idxmax()方法返回轴上最大值第一次出现的索引，是我们自己定义的“left”和“right”
        # state_actions中是动作价值，那么我们找到价值最大的那个，它对应的索引就是left或者right，向左或者向右
        # .argmax返回的是最大值的int位置，也就是下表，是数字，而不是索引
        # 详情见https://blog.csdn.net/m0_37690430/article/details/127185238
        # idmax()是返回索引，argmax()是返回下标，注意区别
    return action_name

def get_env_feedback(S, A):
# 输入为现在的状态S和将要进行的动作A
    if A == 'right':
        if S == N_STATES -2:
        # S=4，再向右走即走到了5，找到了宝藏
            S_ = 'terminal'
            R = 1
        else:
            S_ = S + 1
            R = 0
    else:
    # 向左走
        R = 0
        if S == 0:
            S_ = S
            # 已经在最左边了
        else:
            S_ = S -1
    return S_, R


def update_env(S, episode, step_counter):
    # This is how environment be updated
    env_list = ['-']*(N_STATES-1) + ['T']   # '---------T' our environment
    if S == 'terminal':
        interaction = 'Episode %s: total_steps = %s' % (episode+1, step_counter)
        print('\r{}'.format(interaction), end='')
        time.sleep(2)
        print('\r                                ', end='')
    else:
        env_list[S] = 'o'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction), end='')
        time.sleep(FRESH_TIME)

# 模型训练主体
def rl():
    q_table = build_q_table(N_STATES, ACTIONS)
    # 初始化一个价值表
    for episode in range(MAX_EPISODES):
        # 一轮一轮地跑，最多跑MAX_EPISONDES轮
        step_counter = 0
        # 记录步数
        S = 0
        # 初始化状态位置
        is_terminated = False
        # "结束了"，作为对本轮游戏是否结束的判断，也是退出循环的标志
        update_env(S, episode, step_counter)
        while not is_terminated:
            # while not，当后面的值是0或者false的时候就会一直循环

            A = choose_action(S, q_table)
            # 根据现有状态和价值表选择动作
            S_, R = get_env_feedback(S, A)
            # 根据S做出a后，环境会返回一个新的状态S_以及奖励R
            # ###########集齐了s，a，s_,r四元素后就要开始更新DQN了（这里是表格）##########
            q_predict = q_table.loc[S, A]
            # 可以理解为DQN对状态S下做状态A价值的估计，是我们要更新的量，希望它能更接近真实
            ########TD_learning########
            if S_  != 'terminal':
                # 游戏尚未结束，继续更新参数
                q_target = R + LAMBDA * q_table.iloc[S_, :].max()
                # R 是环境返回的奖励，即真实价值
                # LAMBDA 是衰减因子，后续的奖励预测不及现在的，所以衰减
                # q_table.iloc[S_, :].max()，这个是算法推导出来的
                # 最优价值函数的关键就在于是在S_t+1时刻的价值预测进行最大化处理(对动作a求最大化)
            else:
                q_target = R
                is_terminated = True
                # 这一句会直接跳出while循环，进行下一轮的游戏了
                #游戏结束了

            # #######更新价值表，即更新DQN网络##########

            q_table.loc[S, A] += ALPHA * (q_target - q_predict)
            # 目标就是想让价值表对于动作价值的评估更接近于target，进而更接近真实值
            S = S_
            # 进入下一个状态

            update_env(S, episode, step_counter+1)
            step_counter += 1
    return q_table
# 返回训练好的价值表，即DQN神经网络

In [8]:
if __name__ == "__main__":
    q_table = rl()
    print('\r\nQ-table:\n')
    print(q_table)

                                
Q-table:

       left     right
0  0.000001  0.005728
1  0.000271  0.032612
2  0.002454  0.111724
3  0.000073  0.343331
4  0.000810  0.745813
5  0.000000  0.000000
